In [1]:
import numpy as np
from model.layers import Embedding, Sigmoid, Softmax, Linear
from utils import _Huffman_Tree, corpus_making, delete_low_freq, train_data_gen

In [46]:
import numpy as np
from model.layers import Embedding, Sigmoid, Softmax, Linear
from utils import _Huffman_Tree

class BCELoss:
    def __init__(self):
        self.params = None
        self.grads = None
        self.eps = 1e-8

        self.y_pred , self.target = None, None
        self.loss = None

    def forward(self, y_pred, target, dim = 1):

        self.y_pred = y_pred
        self.target = target
        self.dim = dim
        
        self.loss = -self.target * np.log(self.y_pred + self.eps) - (1 - self.target) * np.log(1 - self.y_pred + self.eps)
        self.loss = np.sum(self.loss, axis = dim)

        return self.loss

    def backward(self, dout = 1):
        dx = (self.y_pred - self.target) / (self.y_pred * (1 - self.y_pred) + + self.eps) 
        return dx * dout

class Hsoftmax:
    def __init__(self, vocab_size, projection, sample_size):
        self.Embedding = Embedding(vocab_size, projection)
        self.HSvector = Embedding(vocab_size - 1 , projection)
        self.sigmoid = Sigmoid()
        self.sample_size = sample_size

        self.layers = [self.Embedding, self.HSvector]

        self.params = []
        self.grads = []

        for layer in self.layers:
            self.params.append(layer.params)
            self.grads.append(layer.grads)

    def forward(self, x, label):
        '''
        inputs : 1 x D(projection)
        label : 1 x [direction_path(1, depth), idx_path(1, depth)]
        label 과 output 의 argmax를 비교해서 같으면 1 틀리면 0 을 부여한 후 이를 target vector로 설정해야됨
        '''

        dir_path = np.array(label[0])
        idx_path = np.expand_dims(label[1], 1)
        self.x = x
        
        self.hidden = self.Embedding.forward(x)

        self.hirearchy_vectors = self.HSvector.forward(dir_path)

        out = np.matmul(self.hirearchy_vectors , self.hidden.T )

        out = self.sigmoid.forward(out)

        mask = np.zeros_like(out)
        mask[mask >= 0.5] = 1

        target = np.zeros_like(out)
        target[mask == idx_path] = 1

        return out , target

    def backward(self, dout):

        W_in, W_out = self.params
        #length x 1
        d_sig = self.sigmoid.backward(dout)

        #vocab -1 x hidden
        d_lin = np.matmul(d_sig , self.hidden)
        d_h = np.matmul(dout.T, self.hirearchy_vectors)

        self.HSvector.backward(d_lin)
        self.Embedding.backward(d_h)

In [3]:
path = "./data/text8.txt"

In [4]:
word2idx , idx2word = corpus_making(path, batch= 50000)
word2idx, idx2word = delete_low_freq(word2idx, idx2word, 10)

MAKING CORPUS: 100%|██████████████████████████████████████████████████████████████| 2000/2000 [00:10<00:00, 197.23it/s]


In [5]:
train_set_idx, total_len = train_data_gen(path = path, word2idx = word2idx, max_distance= 3, batch = 50000)

Making Train_set: 100%|████████████████████████████████████████████████████████████| 2000/2000 [00:38<00:00, 51.77it/s]


In [6]:
b_tree, max_ = _Huffman_Tree(word2idx)

Huffman_Tree: 100%|██████████████████████████████████████████████████████████| 47158/47158 [00:00<00:00, 194584.73it/s]


In [47]:
total_num = len(word2idx)
model = Hsoftmax(total_num, 200, 3)
criterion = BCELoss()

In [8]:
label = np.array([[path, idx_path] for _, _, idx_path ,path in b_tree])

In [53]:
batch_train = train_set_idx[np.random.choice(100, 1)]

x_train = batch_train[:,3]
label_train_idx = np.delete(batch_train , 3)

#N x [path(2C), idx_path(2C)]
label_train = label[np.random.choice(label_train_idx, 3)]

In [60]:
for x, lab in zip(repeat(x_train), label_train):
    print(x)
    x = [x]
    y, t = model.forward(x, lab)
    loss = criterion.forward(y, t, dim = 0)
    dloss = criterion.backward()
    model.backward(dloss)

[16]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])]
[16]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[array([[0., 0., 0., ..., 0., 0., 0.

In [55]:
label_train

array([[array([47157, 47155, 47152, 47145, 47132, 47107, 47065, 46991, 46859,
       46609, 46149, 45345]),
        array([0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1])],
       [array([47157, 47155, 47152, 47146, 47134, 47112, 47075, 47004, 46881,
       46647, 46218, 45467, 44196, 42198, 39199, 34931, 29140]),
        array([0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1])],
       [array([47157, 47156, 47154, 47149, 47139, 47121, 47091, 47035, 46936,
       46754, 46417]),
        array([1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1])]], dtype=object)

In [58]:
from itertools import repeat